In [1]:
from vnstock import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bt
from backtesting import Backtest, Strategy
from backtesting.test import SMA
from backtesting.lib import crossover
from backtesting.lib import resample_apply
%matplotlib inline

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [3]:
params = {"exchangeName": "HOSE,HNX,UPCOM"}
df = stock_screening_insights(params, size=1700, drop_lang='vi')
listing_df = listing_companies()
df = df.merge(listing_df, how='left', on='ticker')

# Filter for stocks in VN100 index
df = df[df['VN100']==True]

# Filter for stocks meeting conditions: Price above SMA(50), Price above SMA(100)
df = df[(df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)') & (df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)')]

df

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,...,VNCOND,VNCONS,VNENE,VNFIN,VNHEAL,VNIND,VNIT,VNMAT,VNREAL,VNUTI
12,ACB,,109336.0,24.8,3.4,4.3,3.6,3.6,0.1,0.8,...,False,False,False,True,False,False,False,False,False,False
102,BID,,308964.0,19.8,3.2,3.3,4.3,3.6,0.1,1.0,...,False,False,False,True,False,False,False,False,False,False
277,CTG,,191709.0,17.1,3.3,3.6,4.5,3.6,0.1,1.0,...,False,False,False,True,False,False,False,False,False,False
311,DCM,,18529.0,10.8,3.1,2.2,4.4,5.0,0.1,1.4,...,False,False,False,False,False,False,False,True,False,False
322,DGC,,46941.0,28.2,3.6,4.5,4.2,4.8,0.3,1.4,...,False,False,False,False,False,False,False,True,False,False
327,DHC,,3542.0,17.2,3.0,3.3,3.5,4.4,0.0,0.8,...,False,False,False,False,False,False,False,True,False,False
502,GMD,,24044.0,28.7,3.2,3.8,4.4,4.6,0.2,1.1,...,False,False,False,False,False,True,False,False,False,False
550,HDB,,68843.0,24.7,3.1,4.7,3.0,2.8,0.1,0.7,...,False,False,False,True,False,False,False,False,False,False
683,IMP,,4851.0,15.0,3.4,4.0,3.0,4.8,0.1,0.4,...,False,False,False,False,True,False,False,False,False,False
701,KBC,,27403.0,11.7,3.7,4.0,3.9,4.8,0.0,1.6,...,False,False,False,False,False,False,False,False,True,False


In [2]:
def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

The strategy roughly goes like this:

Buy a position when:
* daily RSI(14) $>$ 70
* Close $>$ MA(10) $>$ MA(50) $>$ MA(100)
* Volume > MA(20)

Close the position when:
* Daily close is more than 2% _below_ MA(10)
* 5% fixed stop loss is hit

We need to provide bars data in the _lowest time frame_ (i.e. daily) and resample it to any higher time frame (i.e. weekly) that our strategy requires.

In [3]:
class System(Strategy):
    d_rsi = 14  # Daily RSI lookback periods
    w_rsi = 14  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        self.vma20 = self.I(SMA, self.data.Volume, 20)
        # Compute daily RSI(14)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        volume = self.data.Volume[-1]
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and 
            volume > self.vma20[-1] and
            self.daily_rsi[-1] > self.level and
            self.ma10[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 3% fixed stop loss.
            self.buy(sl=.95 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .95 * self.ma10[-1]:
            self.position.close()

In [4]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df[~df.index.duplicated(keep='first')]
    df['Open'] /= 1000
    df['High'] /= 1000
    df['Low'] /= 1000
    df['Close'] /= 1000
    return df

csv_files = [('ACB.csv', 'df_acb'), ('GMD.csv', 'df_gmd'), ('HDB.csv', 'df_hdb'), ('MBB.csv', 'df_mbb'), ('VIX.csv', 'df_vix')]

for file, var_name in csv_files:
    df = process_csv(file)
    exec(f"{var_name} = df")

# ACB

In [8]:
backtest_acb = Backtest(df_acb, System, commission=.00127)
acb = backtest_acb.run()
backtest_acb.plot()
acb

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                    34.95935
Equity Final [$]                 15743.825679
Equity Peak [$]                  16618.325679
Return [%]                          57.438257
Buy & Hold Return [%]              225.539568
Return (Ann.) [%]                   12.325784
Volatility (Ann.) [%]               17.817786
Sharpe Ratio                         0.691769
Sortino Ratio                        1.168021
Calmar Ratio                           0.6087
Max. Drawdown [%]                  -20.249354
Avg. Drawdown [%]                   -4.797069
Max. Drawdown Duration      945 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                      25.344169
Worst Trade [%]                     -5.120497
Avg. Trade [%]                    

In [9]:
%%time
backtest_acb.plot()
backtest_acb.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2 s
Wall time: 1.98 s


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   51.727642
Equity Final [$]                 17920.747932
Equity Peak [$]                  18915.247932
Return [%]                          79.207479
Buy & Hold Return [%]              225.539568
Return (Ann.) [%]                   16.113813
Volatility (Ann.) [%]               21.314093
Sharpe Ratio                         0.756017
Sortino Ratio                        1.325933
Calmar Ratio                         0.539905
Max. Drawdown [%]                  -29.845626
Avg. Drawdown [%]                   -4.712608
Max. Drawdown Duration      998 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                       32.03671
Worst Trade [%]                     -5.936314
Avg. Trade [%]                    

# GMD 

In [11]:
backtest_gmd = Backtest(df_gmd, System, commission=.00127)
gmd = backtest_gmd.run()
backtest_gmd.plot()
gmd

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   18.503539
Equity Final [$]                  8566.088373
Equity Peak [$]                  12832.150278
Return [%]                         -14.339116
Buy & Hold Return [%]              399.370277
Return (Ann.) [%]                   -3.866932
Volatility (Ann.) [%]               14.355459
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.104875
Avg. Drawdown [%]                  -11.296889
Max. Drawdown Duration     1185 days 00:00:00
Avg. Drawdown Duration      256 days 00:00:00
# Trades                                    9
Win Rate [%]                        22.222222
Best Trade [%]                      20.991535
Worst Trade [%]                     -6.258726
Avg. Trade [%]                    

In [12]:
%%time
backtest_gmd.plot()
backtest_gmd.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 1.7 s
Wall time: 1.74 s


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                    51.56724
Equity Final [$]                 10524.758524
Equity Peak [$]                  15047.496097
Return [%]                           5.247585
Buy & Hold Return [%]              399.370277
Return (Ann.) [%]                    1.311726
Volatility (Ann.) [%]               23.487481
Sharpe Ratio                         0.055848
Sortino Ratio                        0.082959
Calmar Ratio                         0.033228
Max. Drawdown [%]                  -39.476585
Avg. Drawdown [%]                   -9.280413
Max. Drawdown Duration      953 days 00:00:00
Avg. Drawdown Duration      117 days 00:00:00
# Trades                                   16
Win Rate [%]                            31.25
Best Trade [%]                      27.510475
Worst Trade [%]                     -5.808301
Avg. Trade [%]                    

# HDB

In [13]:
backtest_hdb = Backtest(df_hdb, System, commission=.00127)
hdb = backtest_hdb.run()
backtest_hdb.plot()
hdb

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                    19.91911
Equity Final [$]                 13942.256482
Equity Peak [$]                  14265.656482
Return [%]                          39.422565
Buy & Hold Return [%]              247.826087
Return (Ann.) [%]                    8.836978
Volatility (Ann.) [%]               15.189979
Sharpe Ratio                         0.581764
Sortino Ratio                        0.947513
Calmar Ratio                         0.608919
Max. Drawdown [%]                  -14.512569
Avg. Drawdown [%]                   -4.228348
Max. Drawdown Duration      947 days 00:00:00
Avg. Drawdown Duration       76 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      18.349696
Worst Trade [%]                     -5.668933
Avg. Trade [%]                    

In [14]:
%%time
backtest_hdb.plot()
backtest_hdb.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 1.91 s
Wall time: 1.95 s


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   48.028311
Equity Final [$]                 20205.475071
Equity Peak [$]                  20674.075071
Return [%]                         102.054751
Buy & Hold Return [%]              247.826087
Return (Ann.) [%]                   19.628424
Volatility (Ann.) [%]               23.686275
Sharpe Ratio                         0.828683
Sortino Ratio                        1.525285
Calmar Ratio                         0.736712
Max. Drawdown [%]                  -26.643275
Avg. Drawdown [%]                   -3.763754
Max. Drawdown Duration      931 days 00:00:00
Avg. Drawdown Duration       49 days 00:00:00
# Trades                                    9
Win Rate [%]                        44.444444
Best Trade [%]                      38.017138
Worst Trade [%]                     -5.221486
Avg. Trade [%]                    

# MBB

In [15]:
backtest_mbb = Backtest(df_mbb, System, commission=.00127)
mbb = backtest_mbb.run()
backtest_mbb.plot()
mbb

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   37.613751
Equity Final [$]                 21741.408531
Equity Peak [$]                  23692.308531
Return [%]                         117.414085
Buy & Hold Return [%]              228.492393
Return (Ann.) [%]                   21.882641
Volatility (Ann.) [%]               20.836692
Sharpe Ratio                         1.050197
Sortino Ratio                        1.998234
Calmar Ratio                         1.525296
Max. Drawdown [%]                  -14.346492
Avg. Drawdown [%]                   -3.575272
Max. Drawdown Duration      512 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      44.929576
Worst Trade [%]                     -5.465723
Avg. Trade [%]                    

In [16]:
%%time
backtest_mbb.plot()
backtest_mbb.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2.27 s
Wall time: 2.27 s


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   47.927199
Equity Final [$]                 34493.413291
Equity Peak [$]                  37588.813291
Return [%]                         244.934133
Buy & Hold Return [%]              228.492393
Return (Ann.) [%]                   37.093448
Volatility (Ann.) [%]               26.740406
Sharpe Ratio                         1.387168
Sortino Ratio                        3.118859
Calmar Ratio                         2.678525
Max. Drawdown [%]                  -13.848462
Avg. Drawdown [%]                   -3.386133
Max. Drawdown Duration      485 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                      46.155845
Worst Trade [%]                     -6.528965
Avg. Trade [%]                    

# VIX

In [17]:
backtest_vix = Backtest(df_vix, System, commission=.00127)
vix = backtest_vix.run()
backtest_vix.plot()
vix

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   19.755601
Equity Final [$]                 24041.659676
Equity Peak [$]                  27978.619814
Return [%]                         140.416597
Buy & Hold Return [%]             1006.936416
Return (Ann.) [%]                   25.245684
Volatility (Ann.) [%]               33.757222
Sharpe Ratio                          0.74786
Sortino Ratio                        1.559881
Calmar Ratio                         1.199127
Max. Drawdown [%]                   -21.05338
Avg. Drawdown [%]                   -7.950935
Max. Drawdown Duration      544 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                   13
Win Rate [%]                        46.153846
Best Trade [%]                       38.02549
Worst Trade [%]                     -6.885697
Avg. Trade [%]                    

In [18]:
%%time
backtest_vix.plot()
backtest_vix.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2.09 s
Wall time: 2.11 s


Start                     2020-05-04 00:00:00
End                       2024-04-11 00:00:00
Duration                   1438 days 00:00:00
Exposure Time [%]                   29.124236
Equity Final [$]                 64193.487866
Equity Peak [$]                  69308.487866
Return [%]                         541.934879
Buy & Hold Return [%]             1006.936416
Return (Ann.) [%]                     61.1453
Volatility (Ann.) [%]               49.594081
Sharpe Ratio                         1.232915
Sortino Ratio                         3.64613
Calmar Ratio                         2.478662
Max. Drawdown [%]                  -24.668671
Avg. Drawdown [%]                   -7.180905
Max. Drawdown Duration      548 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                       78.75026
Worst Trade [%]                     -6.127801
Avg. Trade [%]                    